In [19]:

# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator



In [20]:

# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)


In [21]:

# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')


Found 28709 images belonging to 7 classes.


In [22]:

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')


Found 7178 images belonging to 7 classes.


In [37]:
import tensorflow as tf
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001, decay=1e-6)
, metrics=['accuracy'])
#decay

In [38]:

# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)


C:\Users\91756\AppData\Local\Temp\ipykernel_35336\2106188057.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/50
448/448 [==============================] - 278s 618ms/step - loss: 1.6904 - accuracy: 0.3197 - val_loss: 1.4921 - val_accuracy: 0.4290
Epoch 2/50
448/448 [==============================] - 144s 321ms/step - loss: 1.4092 - accuracy: 0.4571 - val_loss: 1.2979 - val_accuracy: 0.4992
Epoch 3/50
448/448 [==============================] - 141s 315ms/step - loss: 1.2758 - accuracy: 0.5124 - val_loss: 1.2229 - val_accuracy: 0.5393
Epoch 4/50
448/448 [==============================] - 138s 307ms/step - loss: 1.2107 - accuracy: 0.5371 - val_loss: 1.1463 - val_accuracy: 0.5665
Epoch 5/50
448/448 [==============================] - 142s 317ms/step - loss: 1.1447 - accuracy: 0.5629 - val_loss: 1.1301 - val_accuracy: 0.5660
Epoch 6/50
448/448 [==============================] - 4890s 11s/step - loss: 1.0975 - accuracy: 0.5842 - val_loss: 1.1112 - val_accuracy: 0.5780
Epoch 7/50
448/448 [==============================] - 150s 335ms/step - loss: 1.0641 - accuracy: 0.5975 - val_loss: 1.0841 - 

In [39]:

# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)


In [40]:

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')
